# What is about ?

Surprise: Lightgbm boosting shows bad perforamce with default params - worse than constant prediction.
At least for some targets in MoA kaggle competition. 

For some targets curing is simple - turn on "DART mode":  model = lgb.LGBMClassifier(**{ 'boosting_type': 'dart'}).
However it would not work for targets with high disbalance. 

Nevertheless even when Dart works, my tuning of params for DART mode gives performace WORSE than appropriate logreg. 

Challenges : can one explain what causes poor performance of LGB ? Can one better tune params to get better performance than log reg ? 



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load data and small preprocess

In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import datetime
import time

import matplotlib.pyplot as plt
import seaborn as sns

import time
df = pd.read_csv('/kaggle/input/lish-moa/train_features.csv',index_col = 0)  
df0 = df.copy()
df['cp_type'] = df['cp_type'].map({'trt_cp':1.0, 'ctl_vehicle':1.0}) # Forget about control group  
df['cp_dose'] = df['cp_dose'].map({'D1':0.0, 'D2':1.0})
df['cp_time'] = df['cp_time'].map({24:0.0, 48: .5 , 72:1.0})
X = df.copy()
X_save = X.copy()
df_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv',index_col = 0)
df0_test = df_test.copy()
df_test['cp_type'] = df_test['cp_type'].map({'trt_cp':1.0, 'ctl_vehicle':0.0})
df_test['cp_dose'] = df_test['cp_dose'].map({'D1':0.0, 'D2':1.0})
df_test['cp_time'] = df_test['cp_time'].map({24:0.0, 48: .5 , 72:1.0})

y = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv',index_col = 0 )
y_save = y.copy()
print(y.iloc[:3,:2])
df

In [ ]:
y_save.sum(axis = 0 ).sort_values(ascending = False)


# Show LGB default params

In [ ]:
model = lgb.LGBMClassifier(**{}) # {'boosting_type': 'dart'})
model.get_params()

In [ ]:
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression


# Compare LGB different params, constant predict, logreg 

In [ ]:
%%time 
rs = 0
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state= rs )

target_name = 'dopamine_receptor_antagonist'
y = y_save[target_name]
print('Target:', target_name, ' ,  count non-zero:', y.sum() )
print()

print('Predict by constant ' )
print('Logloss: ', np.round( -log_loss( y, np.ones_like(y)*y.mean() ) ,5 ) )
print()

model = lgb.LGBMClassifier(**{}) # {'boosting_type': 'dart'})
t0 = time.time()
cv_score = cross_val_score(model, X, y, cv=skf,   scoring='neg_log_loss' )
print('LGB default params - WORSE than constant!!!')
print('Logloss CV3', np.round( np.mean(cv_score), 5), 'Std', np.round( np.std(cv_score),5),'Folds:',   
      np.round(cv_score ,5), np.round(time.time()-t0, 1), 'seconds passed' )  
print()

model = lgb.LGBMClassifier(**{ 'boosting_type': 'dart'})
t0 = time.time()
cv_score = cross_val_score(model, X, y, cv=skf,   scoring='neg_log_loss' )
print('LGB DART - not bad')
print('Logloss CV3', np.round( np.mean(cv_score), 5), 'Std', np.round( np.std(cv_score),5),'Folds:',   np.round(cv_score ,5),
     np.round(time.time()-t0, 1), 'seconds passed' )
print()

model = lgb.LGBMClassifier(**{ 'boosting_type': 'dart', 'learning_rate':0.100028})
t0 = time.time()
cv_score = cross_val_score(model, X, y, cv=skf,   scoring='neg_log_loss' )
print('LGB DART , learning_rate=0.100028 - small improvement')
print('Logloss CV3', np.round( np.mean(cv_score), 5), 'Std', np.round( np.std(cv_score),5),'Folds:',   
      np.round(cv_score ,5), np.round(time.time()-t0, 1), 'seconds passed' )
print()

model = lgb.LGBMClassifier(**{ 'boosting_type': 'dart', 'learning_rate':0.100028,
 'max_depth':15, 'n_estimators':122, 'num_leaves':27, 'reg_alpha':0.0009} )
t0 = time.time()
cv_score = cross_val_score(model, X, y, cv=skf,   scoring='neg_log_loss' )
print('Best params')
print('Logloss CV3', np.round( np.mean(cv_score), 5), 'Std', np.round( np.std(cv_score),5),'Folds:',   
      np.round(cv_score ,5), np.round(time.time()-t0, 1), 'seconds passed' )
print()

model = LogisticRegression(C = 0.003)
t0 = time.time()
cv_score = cross_val_score(model, X, y, cv=skf,   scoring='neg_log_loss' )
print('Logreg')
print('Logloss CV3', np.round( np.mean(cv_score), 5), 'Std', np.round( np.std(cv_score),5),'Folds:',   
      np.round(cv_score ,5), np.round(time.time()-t0, 1), 'seconds passed' )
print()

    


In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C = 0.002)
t0 = time.time()
cv_score = cross_val_score(model, X, y, cv=skf,   scoring='neg_log_loss' )
print('Logreg')
print('Logloss CV3', np.round( np.mean(cv_score), 5), 'Std', np.round( np.std(cv_score),5),'Folds:',   
      np.round(cv_score ,5), np.round(time.time()-t0, 1), 'seconds passed' )
print()


# Example where Dart is worse than constant

That would be typical if number disbalance is very high, however in that example 93 samples  = 1 , it is not so low, for many other targets which even lower number Lgb is better than constant 

In [ ]:
%%time 
rs = 0
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state= rs )

target_name = 'jak_inhibitor'
y = y_save[target_name]
print('Target:', target_name, ' ,  count non-zero:', y.sum() )
print()

print('Predict by constant ' )
print('Logloss: ', np.round( -log_loss( y, np.ones_like(y)*y.mean() ) ,5 ) )
print()

model = lgb.LGBMClassifier(**{}) # {'boosting_type': 'dart'})
t0 = time.time()
cv_score = cross_val_score(model, X, y, cv=skf,   scoring='neg_log_loss' )
print('LGB default params - WORSE than constant!!!')
print('Logloss CV3', np.round( np.mean(cv_score), 5), 'Std', np.round( np.std(cv_score),5),'Folds:',   
      np.round(cv_score ,5), np.round(time.time()-t0, 1), 'seconds passed' )  
print()

model = lgb.LGBMClassifier(**{ 'boosting_type': 'dart'})
t0 = time.time()
cv_score = cross_val_score(model, X, y, cv=skf,   scoring='neg_log_loss' )
print('LGB DART - again WORSE than constant!!!')
print('Logloss CV3', np.round( np.mean(cv_score), 5), 'Std', np.round( np.std(cv_score),5),'Folds:',   np.round(cv_score ,5),
     np.round(time.time()-t0, 1), 'seconds passed' )
print()

model = lgb.LGBMClassifier(**{ 'boosting_type': 'dart', 'learning_rate':0.100028})
t0 = time.time()
cv_score = cross_val_score(model, X, y, cv=skf,   scoring='neg_log_loss' )
print('LGB DART , learning_rate=0.100028 - small change of LR, but big improvement , but  again WORSE than constant!!!')
print('Logloss CV3', np.round( np.mean(cv_score), 5), 'Std', np.round( np.std(cv_score),5),'Folds:',   
      np.round(cv_score ,5), np.round(time.time()-t0, 1), 'seconds passed' )
print()

model = lgb.LGBMClassifier(**{ 'boosting_type': 'dart', 'learning_rate':0.100028,
 'max_depth':15, 'n_estimators':122, 'num_leaves':27, 'reg_alpha':0.0009} )
t0 = time.time()
cv_score = cross_val_score(model, X, y, cv=skf,   scoring='neg_log_loss' )
print('Best previous params - again WORSE than constant!!! ')
print('Logloss CV3', np.round( np.mean(cv_score), 5), 'Std', np.round( np.std(cv_score),5),'Folds:',   
      np.round(cv_score ,5), np.round(time.time()-t0, 1), 'seconds passed' )
print()

model = LogisticRegression(C = 0.02)
t0 = time.time()
cv_score = cross_val_score(model, X, y, cv=skf,   scoring='neg_log_loss' )
print('Logreg')
print('Logloss CV3', np.round( np.mean(cv_score), 5), 'Std', np.round( np.std(cv_score),5),'Folds:',   
      np.round(cv_score ,5), np.round(time.time()-t0, 1), 'seconds passed' )
print()

    
